<a href="https://colab.research.google.com/github/HaoranYueRyan/Data-Science-analysis-Approaches/blob/master/assignment_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install 'aif360[LFR]'

     |████████████████████████████████| 175 kB 5.0 MB/s 
     |████████████████████████████████| 564 kB 55.0 MB/s 
  Created wheel for memory-profiler: filename=memory_profiler-0.60.0-py3-none-any.whl size=31284 sha256=494953283bf6461105318bed0bebf796c21f1ab00fb3d6265c6cd09ec1a34205
  Stored in directory: /root/.cache/pip/wheels/67/2b/fb/326e30d638c538e69a5eb0aa47f4223d979f502bbdb403950f
Successfully built memory-profiler


In [2]:
cd /usr/local/lib/python3.7/dist-packages/aif360/data/raw/adult

/usr/local/lib/python3.7/dist-packages/aif360/data/raw/adult


In [3]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test

--2022-04-24 10:45:18--  https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3974305 (3.8M) [application/x-httpd-php]
Saving to: ‘adult.data’

adult.data          100%[===================>]   3.79M  7.42MB/s    in 0.5s    

2022-04-24 10:45:19 (7.42 MB/s) - ‘adult.data’ saved [3974305/3974305]

--2022-04-24 10:45:19--  https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5229 (5.1K) [application/x-httpd-php]
Saving to: ‘adult.names’

adult.names         100%[===================>]   5.11K  --.-KB/s    in 0s      

2022-

In [47]:
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult

privileged_groups=[{'sex':1}]
unprivileged_groups = [{'sex': 0}]
dataset_orig = load_preproc_data_adult(['sex'])


#STEP 3: We split between training and test set.
train, test = dataset_orig.split([0.7], shuffle=True)
print("training data size", train.features.shape)
print("dataset feature names", train.feature_names)

training data size (34189, 18)
dataset feature names ['race', 'sex', 'Age (decade)=10', 'Age (decade)=20', 'Age (decade)=30', 'Age (decade)=40', 'Age (decade)=50', 'Age (decade)=60', 'Age (decade)=>=70', 'Education Years=6', 'Education Years=7', 'Education Years=8', 'Education Years=9', 'Education Years=10', 'Education Years=11', 'Education Years=12', 'Education Years=<6', 'Education Years=>12']


In [6]:
from sklearn.preprocessing import StandardScaler 


#Normalize the dataset, both train and test. This should always be done in any machine learning pipeline!
scale_orig = StandardScaler()
X_train = scale_orig.fit_transform(train.features)
y_train = train.labels.ravel()

X_test = scale_orig.transform(test.features) 
y_test = test.labels.ravel()

In [57]:
def get_score(model,X_train,X_test,y_train,y_test):
  model.fit(X_train,y_train)
  return model.score(X_test,y_test)

In [41]:
from sklearn.model_selection import GridSearchCV 
from sklearn.linear_model import LogisticRegression
import pandas as pd


clf= GridSearchCV(LogisticRegression(),{'solver':['saga','liblinear'],
'C':[1.0, 100000., 0.001, 0.000001]},cv=5,return_train_score=False)

clf.fit(X_train,y_train)
clf.cv_results_
df=pd.DataFrame(clf.cv_results_)
# df[['param_C','param_solver','mean_test_score']]

The max_iter was reached which means the coef_ did not converge


In [42]:
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.280826,0.108783,0.003203,0.000623,1.0,saga,"{'C': 1.0, 'solver': 'saga'}",0.806084,0.802720,0.799210,0.806376,0.799912,0.802860,0.002993,1
1,0.169267,0.026867,0.002272,0.000225,1.0,liblinear,"{'C': 1.0, 'solver': 'liblinear'}",0.806084,0.802720,0.799210,0.806376,0.799912,0.802860,0.002993,1
2,0.402361,0.364304,0.002398,0.000535,100000.0,saga,"{'C': 100000.0, 'solver': 'saga'}",0.806084,0.802720,0.799210,0.806376,0.799912,0.802860,0.002993,1
3,0.161299,0.022079,0.002106,0.000008,100000.0,liblinear,"{'C': 100000.0, 'solver': 'liblinear'}",0.806084,0.802720,0.799210,0.806376,0.799912,0.802860,0.002993,1
4,0.211196,0.009608,0.003456,0.001116,0.001,saga,"{'C': 0.001, 'solver': 'saga'}",0.805645,0.802866,0.799210,0.806230,0.799620,0.802714,0.002927,6
5,0.078854,0.003023,0.002237,0.000097,0.001,liblinear,"{'C': 0.001, 'solver': 'liblinear'}",0.805645,0.802720,0.799210,0.806230,0.799766,0.802714,0.002895,5
6,0.201524,0.008101,0.002083,0.000131,0.000001,saga,"{'C': 1e-06, 'solver': 'saga'}",0.761480,0.761626,0.761626,0.761626,0.761591,0.761590,0.000057,8
7,0.039517,0.001572,0.002140,0.000009,0.000001,liblinear,"{'C': 1e-06, 'solver': 'liblinear'}",0.797309,0.793946,0.794969,0.798187,0.793769,0.795636,0.001794,7


In [43]:
df[['param_C','param_solver','mean_test_score']]

,param_C,param_solver,mean_test_score
0,1.0,saga,0.802860
1,1.0,liblinear,0.802860
2,100000.0,saga,0.802860
3,100000.0,liblinear,0.802860
4,0.001,saga,0.802714
5,0.001,liblinear,0.802714
6,0.000001,saga,0.761590
7,0.000001,liblinear,0.795636


In [44]:
dir(clf)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_check_feature_names',
 '_check_n_features',
 '_check_refit_for_multimetric',
 '_estimator_type',
 '_format_results',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_pairwise',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_required_parameters',
 '_run_search',
 '_select_best_index',
 '_validate_data',
 'best_estimator_',
 'best_index_',
 'best_params_',
 'best_score_',
 'classes_',
 'cv',
 'cv_results_',
 'decision_function',
 'error_score',
 'estimator',
 'fit',
 'get_params',
 'inverse_transform',
 'multimetric_',
 'n_features_

In [45]:
clf.best_score_

0.8028604829553148

In [46]:
clf.best_params_

{'C': 1.0, 'solver': 'saga'}

In [ ]:
from sklearn.model_selection import GridSearchCV 
from sklearn.linear_model import LogisticRegression
import pandas as pd
from aif360.metrics import ClassificationMetric


mlf= GridSearchCV(LogisticRegression(),{'solver':['saga','liblinear'],
'C':[1.0, 100000., 0.001, 0.000001]},cv=5,return_train_score=False)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.neural_network import MLPClassifier
logreg=LogisticRegression()
clf=svm.SVC()
mlp=MLPClassifier()